## Machine Learning

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
import gc


/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print('loading files...')
train = pd.read_csv('../../data/train_prepared.csv', na_values=-1) #train_prepared
test = pd.read_csv('../../data/test_prepared.csv', na_values=-1)

print('files loaded...')

loading files...
files loaded...


In [3]:
#--- memory consumed by train dataframe ---
mem = train.memory_usage(index=True).sum()
print("Memory consumed by training set  :   {} MB" .format(mem/ 1024**2))
 
#--- memory consumed by test dataframe ---
mem = test.memory_usage(index=True).sum()
print("Memory consumed by test set      :   {} MB" .format(mem/ 1024**2))

Memory consumed by training set  :   217.9732208251953 MB
Memory consumed by test set      :   320.1474151611328 MB


In [4]:
def change_datatype(df):
    float_cols = list(df.select_dtypes(include=['int']).columns)
    for col in float_cols:
        if ((np.max(df[col]) <= 127) and(np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767) and(np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647) and(np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)

change_datatype(train)
change_datatype(test) 

In [5]:
#--- Converting columns from 'float64' to 'float32' ---
def change_datatype_float(df):
    float_cols = list(df.select_dtypes(include=['float']).columns)
    for col in float_cols:
        df[col] = df[col].astype(np.float32)
        
change_datatype_float(train)
change_datatype_float(test)

In [6]:
#--- memory consumed by train dataframe ---
mem = train.memory_usage(index=True).sum()
print("Memory consumed by training set  :   {:.3f} MB" .format(mem/ 1024**2))
 
#--- memory consumed by test dataframe ---
mem = test.memory_usage(index=True).sum()
print("Memory consumed by test set      :   {:.3f} MB" .format(mem/ 1024**2))

Memory consumed by training set  :   54.493 MB
Memory consumed by test set      :   80.888 MB


In [ ]:
#test.drop(['ps_car_03_cat', 'ps_car_05_cat'], inplace=True, axis=1)

col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(col_to_drop, axis=1)  
test = test.drop(col_to_drop, axis=1)  

for c in train.select_dtypes(include=['float64']).columns:
    train[c]=train[c].astype(np.float32)
    test[c]=test[c].astype(np.float32)
for c in train.select_dtypes(include=['int64']).columns[2:]:
    train[c]=train[c].astype(np.int8)
    test[c]=test[c].astype(np.int8)    

print(train.shape, test.shape)

In [7]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return [('gini', gini_score)]

In [8]:
# Set xgboost parameters
params = {}
params['eta'] = 0.02 #0.02
params['max_depth'] = 4 # 4
params['subsample'] = 0.9
params['colsample_bytree'] = 0.9
params['silent'] = True
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'auc'
params['silent'] = True
params['seed']: 6
#'maximize':True

#'eta': 0.09, 'max_depth': 5

In [ ]:
X = train.drop(['id', 'target'], axis=1)
features = X.columns
X = X.values
y = train['target'].values
sub=test['id'].to_frame()
sub['target']=0

nrounds=2000  # need to change to 2000
kfold = 5  # need to change to 5

skf = StratifiedKFold(n_splits=kfold, random_state=0)

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    
    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
    
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    
    d_train = xgb.DMatrix(X_train, y_train) 
    d_valid = xgb.DMatrix(X_valid, y_valid) 
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    
    xgb_model = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=100, 
                          feval=gini_xgb, maximize=True, verbose_eval=100)
    
    sub['target'] += xgb_model.predict(xgb.DMatrix(test[features].values), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (2*kfold)
    
sub.to_csv('submission_6.csv', index=False, float_format='%.5f')
gc.collect()
sub.head(2)


 xgb kfold: 1  of  5 : 
[0]	train-gini:0.178181	valid-gini:0.179447
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-gini:0.242031	valid-gini:0.234424
[200]	train-gini:0.273032	valid-gini:0.253313
[300]	train-gini:0.294569	valid-gini:0.265997
[400]	train-gini:0.308457	valid-gini:0.271572
[500]	train-gini:0.319406	valid-gini:0.274661
[600]	train-gini:0.329299	valid-gini:0.276471
[700]	train-gini:0.33746	valid-gini:0.276355
Stopping. Best iteration:
[653]	train-gini:0.33372	valid-gini:0.276743

 xgb kfold: 2  of  5 : 
[0]	train-gini:0.181392	valid-gini:0.172635
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-gini:0.24256	valid-gini:0.227125
[200]	train-gini:0.270398	valid-gini:0.252532
[300]	train-gini:0.292857	valid-gini:0.268609
[400]	train-gini:0.308133	valid-gini:0.274

In [ ]:
xgb_model.best_score #before 0.276476